In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
from functools import partial

import torch
from transformers import BertTokenizer, BertForNextSentencePrediction

from src.language_model_usage.winograd_schema_challenge import get_probability_of_next_sentence

I0201 18:21:05.290322 140609710577408 file_utils.py:35] PyTorch version 1.0.1.post2 available.
W0201 18:21:06.074531 140609710577408 __init__.py:28] To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


ImportError: cannot import name 'get_probability_of_next_sentence'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

partial_get_probability_of_next_sentence = partial(get_probability_of_next_sentence, tokenizer, model)

In [ ]:
text1 = "How old are you?"
text2 = "The Eiffel Tower is in Paris"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

In [ ]:
text1 = "How old are you?"
text2 = "I am 22 years old"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)
print(prediction[0])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-multilingual-cased')

partial_get_probability_of_next_sentence = partial(get_probability_of_next_sentence, tokenizer, model)

In [ ]:
text1 = "Quantos anos você tem?"
text2 = "A Torre Eiffel fica em Paris"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

In [ ]:
text1 = "Quantos anos você tem?"
text2 = "Eu tenho 22 anos"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

In [ ]:
def get_sentence_breaks(first_sentence, second_sentence):
    for i in range(len(first_sentence.split())):
        if first_sentence.split()[i] != second_sentence.split()[i]:  # noqaE226
            break
    return i

In [ ]:
def test_get_sentence_breaks():
    first_sentence = 'The city councilmen refused the demonstrators a permit because the city councilmen feared violence.'
    second_sentence = 'The city councilmen refused the demonstrators a permit because the demonstrators feared violence.'
    i = get_sentence_breaks(first_sentence, second_sentence)
    assert ' '.join(first_sentence.split()[:i]) == \
        'The city councilmen refused the demonstrators a permit because the'
    assert ' '.join(second_sentence.split()[:i]) == \
        'The city councilmen refused the demonstrators a permit because the'
    assert ' '.join(first_sentence.split()[i:]) == \
        'city councilmen feared violence.'
    assert ' '.join(second_sentence.split()[i:]) == \
        'demonstrators feared violence.'
    
    first_sentence = 'Os vereadores recusaram a autorização aos manifestantes porque os vereadores temiam a violência.'
    second_sentence = 'Os vereadores recusaram a autorização aos manifestantes porque os manifestantes temiam a violência.'
    
    i = get_sentence_breaks(first_sentence, second_sentence)
    assert ' '.join(first_sentence.split()[:i]) == \
        'Os vereadores recusaram a autorização aos manifestantes porque os'
    assert ' '.join(second_sentence.split()[:i]) == \
        'Os vereadores recusaram a autorização aos manifestantes porque os'
    assert ' '.join(first_sentence.split()[i:]) == \
        'vereadores temiam a violência.'
    assert ' '.join(second_sentence.split()[i:]) == \
        'manifestantes temiam a violência.'
    
test_get_sentence_breaks()